In [1]:
import pymongo
import json
import pandas as pd
from datetime import datetime, timedelta
from pandas import DataFrame
import matplotlib.pyplot as plt
from datetime import time
from tqdm import tqdm

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["studentlife"]
collection = mydb["activity"]

In [2]:
cursor = collection.find()
documents_list = list(cursor)


df_activity = pd.DataFrame(documents_list)
print(df_activity)

                               _id   timestamp activity inference  uid
0         63645d123ece7167ec10cb74  1364410490                  0  u44
1         63645d123ece7167ec10cb75  1364410498                  0  u44
2         63645d123ece7167ec10cb76  1364410501                  0  u44
3         63645d123ece7167ec10cb77  1364410503                  0  u44
4         63645d123ece7167ec10cb78  1364410506                  0  u44
...                            ...         ...                ...  ...
22711687  63648c673ece7167ec6d56be  1370059187                  0  u49
22711688  63648c673ece7167ec6d56bf  1370059189                  0  u49
22711689  63648c673ece7167ec6d56c0  1370059192                  0  u49
22711690  63648c673ece7167ec6d56c1  1370059195                  0  u49
22711691  63648c673ece7167ec6d56c2  1370059197                  0  u49

[22711692 rows x 4 columns]


In [3]:
df_activity.head(3)

,_id,timestamp,activity inference,uid
0,63645d123ece7167ec10cb74,1364410490,0,u44
1,63645d123ece7167ec10cb75,1364410498,0,u44
2,63645d123ece7167ec10cb76,1364410501,0,u44


In [4]:
morning_start_time = time(6, 0, 0)     # 6:00 AM
afternoon_start_time = time(12, 0, 0)  # 12:00 PM
evening_start_time = time(18, 0, 0)    # 6:00 PM
night_start_time = time(0, 0, 0)       # 12:00 AM (midnight)

In [5]:
def categorize_time_interval(timestamp):
    current_time = timestamp.time()
    
    if morning_start_time <= current_time < afternoon_start_time:
        return "morning"
    elif afternoon_start_time <= current_time < evening_start_time:
        return "afternoon"
    elif evening_start_time <= current_time <= time(23, 59, 59):
        return "evening"
    else:
        return "night"

In [6]:
df_activity = df_activity.drop("_id", axis=1)

In [7]:
df_activity.head(3)

,timestamp,activity inference,uid
0,1364410490,0,u44
1,1364410498,0,u44
2,1364410501,0,u44


In [8]:
df_activity['timestamp'] = pd.to_datetime(df_activity['timestamp'], unit='s')
df_activity['date'] = df_activity['timestamp'].dt.strftime('%Y-%m-%d')

In [9]:
df_activity['time_interval'] = df_activity['timestamp'].apply(categorize_time_interval)
df_activity = df_activity.sort_values(by=['uid', 'timestamp'])

In [10]:
df_activity.head(3)

,timestamp,activity inference,uid,date,time_interval
16525102,2013-03-27 04:00:01,0,u00,2013-03-27,night
16525103,2013-03-27 04:00:04,0,u00,2013-03-27,night
16525104,2013-03-27 04:00:07,0,u00,2013-03-27,night


In [11]:
df_data = DataFrame()

In [12]:
activity_uid = ['u00', 'u01', 'u02', 'u03', 'u04', 'u05', 'u07', 'u08', 'u09', 'u10', 'u12', 'u13', 'u14', 'u15', 'u16',
                'u17', 'u18', 'u19', 'u20', 'u22', 'u23', 'u24', 'u25', 'u27', 'u30', 'u31', 'u32', 'u33', 'u34', 'u35',
                'u36', 'u39', 'u41', 'u42', 'u43', 'u44', 'u45', 'u46', 'u47', 'u49', 'u50', 'u51', 'u52', 'u53', 'u54',
                'u56', 'u57', 'u58', 'u59']

In [13]:
def classify_activity(activity_inference):
    if activity_inference == "0":
        return "stationary"
    elif activity_inference == "1":
        return "walking"
    elif activity_inference == "2":
        return "running"
    elif activity_inference == "3":
        return "unknown"

In [14]:
df_activity['activity'] = df_activity['activity inference'].apply(classify_activity)
df_activity = df_activity.drop("activity inference", axis=1)
df_activity.head(3)

,timestamp,uid,date,time_interval,activity
16525102,2013-03-27 04:00:01,u00,2013-03-27,night,stationary
16525103,2013-03-27 04:00:04,u00,2013-03-27,night,stationary
16525104,2013-03-27 04:00:07,u00,2013-03-27,night,stationary


In [15]:
df_activity['activity'].value_counts()

activity
stationary    20912307
walking         969136
unknown         592360
running         237889
Name: count, dtype: int64

In [16]:
df_activity.head(5)

,timestamp,uid,date,time_interval,activity
16525102,2013-03-27 04:00:01,u00,2013-03-27,night,stationary
16525103,2013-03-27 04:00:04,u00,2013-03-27,night,stationary
16525104,2013-03-27 04:00:07,u00,2013-03-27,night,stationary
16525105,2013-03-27 04:00:09,u00,2013-03-27,night,stationary
16525106,2013-03-27 04:03:12,u00,2013-03-27,night,stationary


In [17]:
df_activity['time_interval'].value_counts()

time_interval
night        5713604
evening      5705619
morning      5698534
afternoon    5593935
Name: count, dtype: int64

In [18]:
def process_activity_data_for_uid(uid, df_activity):
    columns = ['uid', 'date',
               'stationary_duration_morning', 'walking_duration_morning', 'running_duration_morning', 'unknown_duration_morning',
               'stationary_duration_afternoon', 'walking_duration_afternoon', 'running_duration_afternoon', 'unknown_duration_afternoon',
               'stationary_duration_evening', 'walking_duration_evening', 'running_duration_evening', 'unknown_duration_evening',
               'stationary_duration_night', 'walking_duration_night', 'running_duration_night', 'unknown_duration_night']
    result_df = pd.DataFrame(columns=columns)
    user_df = df_activity[df_activity['uid'] == uid]

    for date in user_df['date'].unique():
        date_df = user_df[user_df['date'] == date]

        activities = {
            'stationary_morning': 0, 'walking_morning': 0, 'running_morning': 0, 'unknown_morning': 0,
            'stationary_afternoon': 0, 'walking_afternoon': 0, 'running_afternoon': 0, 'unknown_afternoon': 0,
            'stationary_evening': 0, 'walking_evening': 0, 'running_evening': 0, 'unknown_evening': 0,
            'stationary_night': 0, 'walking_night': 0, 'running_night': 0, 'unknown_night': 0
        }

        grouped_by_interval = date_df.groupby('time_interval')

        for interval, interval_df in grouped_by_interval:
            start_time = None
            previous_activity = None

            first_record = pd.to_datetime(interval_df.iloc[0]['timestamp'])
            init_missing_duration = None

            if interval == "morning":
                manual_start_morning = pd.to_datetime(f"{date} 06:00:00")
                init_missing_duration = (first_record - manual_start_morning).total_seconds()
            elif interval == "afternoon":
                manual_start_afternoon = pd.to_datetime(f"{date} 12:00:00")
                init_missing_duration = (first_record - manual_start_afternoon).total_seconds()
            elif interval == "evening":
                manual_start_evening = pd.to_datetime(f"{date} 18:00:00")
                init_missing_duration = (first_record - manual_start_evening).total_seconds()
            elif interval == "night":
                manual_start_night = pd.to_datetime(f"{date} 00:00:00")
                init_missing_duration = (first_record - manual_start_night).total_seconds()

            duration_hours = init_missing_duration / 3600
            update_variable = f"unknown_{interval}"
            activities[update_variable] += duration_hours

            for index, row in interval_df.iterrows():
                current_activity = row['activity']
                current_time = pd.to_datetime(row['timestamp'])

                if current_activity == previous_activity:
                    if start_time is not None:
                        duration_seconds = (current_time - start_time).seconds
                        duration_hours = duration_seconds / 3600.0

                        update_variable = f"{previous_activity}_{interval}"
                        activities[update_variable] += duration_hours
                    start_time = current_time
                else:
                    if start_time is not None:
                        duration_seconds = (current_time - start_time).seconds
                        duration_hours = duration_seconds / 3600.0
                        update_variable = f"unknown_{interval}"
                        activities[update_variable] += duration_hours
                    start_time = current_time

                previous_activity = current_activity

            last_record = pd.to_datetime(interval_df.iloc[-1]['timestamp'])
            init_missing_duration = None

            if interval == "morning":
                manual_end_morning = pd.to_datetime(f"{date} 12:00:00")
                init_missing_duration = (manual_end_morning - last_record).total_seconds()
            elif interval == "afternoon":
                manual_end_afternoon = pd.to_datetime(f"{date} 18:00:00")
                init_missing_duration = (manual_end_afternoon - last_record).total_seconds()
            elif interval == "evening":
                manual_end_evening = pd.to_datetime(f"{date} 23:59:59")
                init_missing_duration = (manual_end_evening - last_record).total_seconds()
            elif interval == "night":
                manual_end_night = pd.to_datetime(f"{date} 06:00:00")
                init_missing_duration = (manual_end_night - last_record).total_seconds()

            duration_hours = init_missing_duration / 3600
            update_variable = f"unknown_{interval}"
            activities[update_variable] += duration_hours
        
        result_df = pd.concat([result_df, pd.DataFrame([{
            'uid': uid,
            'date': date,
            'stationary_duration_morning': activities['stationary_morning'],
            'walking_duration_morning': activities['walking_morning'],
            'running_duration_morning': activities['running_morning'],
            'unknown_duration_morning': activities['unknown_morning'],
            'stationary_duration_afternoon': activities['stationary_afternoon'],
            'walking_duration_afternoon': activities['walking_afternoon'],
            'running_duration_afternoon': activities['running_afternoon'],
            'unknown_duration_afternoon': activities['unknown_afternoon'],
            'stationary_duration_evening': activities['stationary_evening'],
            'walking_duration_evening': activities['walking_evening'],
            'running_duration_evening': activities['running_evening'],
            'unknown_duration_evening': activities['unknown_evening'],
            'stationary_duration_night': activities['stationary_night'],
            'walking_duration_night': activities['walking_night'],
            'running_duration_night': activities['running_night'],
            'unknown_duration_night': activities['unknown_night'],
        }])], ignore_index=True)


    return result_df

In [19]:
for uid in tqdm(activity_uid):
    print(uid)
    df = process_activity_data_for_uid(str(uid), df_activity)
    frames = [df_data, df]
    df_data = pd.concat(frames, ignore_index=True)

  0%|                                                    | 0/49 [00:00<?, ?it/s]

u00


/var/folders/k8/745_jm4x0n35zzr1p5chpdfh0000gn/T/ipykernel_4443/3350858182.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame([{
  2%|▉                                           | 1/49 [00:11<09:32, 11.92s/it]

u01


/var/folders/k8/745_jm4x0n35zzr1p5chpdfh0000gn/T/ipykernel_4443/3350858182.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame([{
  4%|█▊                                          | 2/49 [00:22<08:42, 11.12s/it]

...

/var/folders/k8/745_jm4x0n35zzr1p5chpdfh0000gn/T/ipykernel_4443/3350858182.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame([{
 96%|█████████████████████████████████████████▏ | 47/49 [08:28<00:21, 10.63s/it]

u58


/var/folders/k8/745_jm4x0n35zzr1p5chpdfh0000gn/T/ipykernel_4443/3350858182.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame([{
 98%|██████████████████████████████████████████ | 48/49 [08:40<00:11, 11.07s/it]

u59


/var/folders/k8/745_jm4x0n35zzr1p5chpdfh0000gn/T/ipykernel_4443/3350858182.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame([{
100%|███████████████████████████████████████████| 49/49 [09:20<00:00, 11.45s/it]


In [20]:
df_data.columns

Index(['uid', 'date', 'stationary_duration_morning',
       'walking_duration_morning', 'running_duration_morning',
       'unknown_duration_morning', 'stationary_duration_afternoon',
       'walking_duration_afternoon', 'running_duration_afternoon',
       'unknown_duration_afternoon', 'stationary_duration_evening',
       'walking_duration_evening', 'running_duration_evening',
       'unknown_duration_evening', 'stationary_duration_night',
       'walking_duration_night', 'running_duration_night',
       'unknown_duration_night'],
      dtype='object')

In [21]:
df_data.isnull().sum()

uid                              0
date                             0
stationary_duration_morning      0
walking_duration_morning         0
running_duration_morning         0
unknown_duration_morning         0
stationary_duration_afternoon    0
walking_duration_afternoon       0
running_duration_afternoon       0
unknown_duration_afternoon       0
stationary_duration_evening      0
walking_duration_evening         0
running_duration_evening         0
unknown_duration_evening         0
stationary_duration_night        0
walking_duration_night           0
running_duration_night           0
unknown_duration_night           0
dtype: int64

In [22]:
len(df_data)

2883

In [23]:
df_data.head(3)

,uid,date,stationary_duration_morning,walking_duration_morning,running_duration_morning,unknown_duration_morning,stationary_duration_afternoon,walking_duration_afternoon,running_duration_afternoon,unknown_duration_afternoon,stationary_duration_evening,walking_duration_evening,running_duration_evening,unknown_duration_evening,stationary_duration_night,walking_duration_night,running_duration_night,unknown_duration_night
0,u00,2013-03-27,5.704444,0.199167,0,0.096389,4.4475,0.981389,0.061667,0.509444,3.984722,0.921944,0.870833,0.222222,1.999167,0,0,4.000833
1,u00,2013-03-28,5.632778,0.129722,0,0.2375,4.8925,0.548611,0.195,0.363889,4.976111,0.470556,0.233333,0.319722,5.68,0.058611,0.192222,0.069167
2,u00,2013-03-29,5.505,0.087222,0.053056,0.354722,4.36,0.958889,0.193889,0.487222,3.996667,1.172778,0.028333,0.801944,5.102778,0.335833,0.053056,0.508333


In [24]:
df_data.to_csv('raw_activity_features_studentlife.csv')

In [25]:
len(df_data['uid'].unique())

49